In [ ]:
import numpy as np

class PendulumDiscretizer:
    def __init__(self,
                 n_theta=42,
                 n_theta_dot=42,
                 n_action=18,
                 theta_dot_max=8.0,
                 u_max=2.0,
                 dt=0.05):
        self.n_theta = n_theta
        self.n_theta_dot = n_theta_dot
        self.n_action = n_action
        self.theta_dot_max = theta_dot_max
        self.u_max = u_max
        self.dt = dt

        # bins for discretization
        self.theta_bins = np.linspace(-np.pi, np.pi, n_theta)
        self.theta_dot_bins = np.linspace(-theta_dot_max,
                                          theta_dot_max,
                                          n_theta_dot)
        self.action_bins = np.linspace(-u_max, u_max, n_action)

    def encode_state(self, state):
        """(theta, theta_dot) → (i,j)"""
        theta, theta_dot = state
        i = np.digitize(theta, self.theta_bins) - 1
        j = np.digitize(theta_dot, self.theta_dot_bins) - 1
        return (np.clip(i, 0, self.n_theta-1),
                np.clip(j, 0, self.n_theta_dot-1))

    def decode_state(self, idx):
        """(i,j) → (theta, theta_dot)"""
        return (self.theta_bins[idx[0]],
                self.theta_dot_bins[idx[1]])

    def decode_action(self, a_idx):
        """a_idx → torque"""
        return float(self.action_bins[a_idx])

def next_state(theta, theta_dot, torque, dt):
    """Pendulum-v1 dynamics model"""
    g = 9.80665
    theta_dd = -3 * g * np.sin(theta + np.pi) / 2 + 3 * torque
    return theta + theta_dot*dt, theta_dot + theta_dd*dt

def cost_fn(theta, theta_dot, torque):
    """cost function from Pendulum-v1"""
    return theta**2 + 0.1*theta_dot**2 + 0.001*torque**2

In [ ]:
if __name__ == "__main__":
    # 1) Discretization
    disc = PendulumDiscretizer(dt=0.1)

    # 2) finite time horizon N
    N = 200

    # 3) DP cost and action storage
    #   C[k,i,j]   = minimum cost at (i,j) at step k
    #   Uopt[k,i,j]= action of minimum cost at (i,j) at step k
    C    = np.zeros((N+1, disc.n_theta, disc.n_theta_dot))
    Uopt = np.zeros((N,   disc.n_theta, disc.n_theta_dot), dtype=int)

    # 4) terminal cost
    # assumption: terminal cost == 0
    C[N, :, :] = 0.0

    # 5) Backward propagation
    for k in range(N-1, -1, -1):
        for i in range(disc.n_theta):
            for j in range(disc.n_theta_dot):
                theta, theta_dot = disc.decode_state((i, j))

                cosmin = np.inf
                best_u = 0

                # discretized action space의 모든 action에 대해 next_state와 cost를 계산
                for a_idx in range(disc.n_action):
                    u = disc.decode_action(a_idx)
                    th2, thdot2 = next_state(theta, theta_dot, u, disc.dt)
                    ni, nj = disc.encode_state((th2, thdot2))
                    r = cost_fn(theta, theta_dot, u)

                    # step k cost + next step cost
                    cost = r + C[k+1, ni, nj]
                    if cost < cosmin:
                        cosmin = cost
                        best_u = a_idx

                C[k, i, j]    = cosmin
                Uopt[k, i, j] = best_u
                print(f"Step {k}, State ({theta},{theta_dot}): Cost={cosmin:.4f}, Action={best_u}")


In [ ]:
import matplotlib.pyplot as plt

# (위에서 C, Uopt, disc, init_state 부분까지 계산된 상태라 가정)

# 초기 상태 인덱스
init_state = (0.0, 0.0)
si, sj = disc.encode_state(init_state)

# 스테이지 0부터 N까지의 cost-to-go
cost_profile = C[:, si, sj]   # 길이가 N+1

# 시각화
plt.figure(figsize=(8,5))
plt.plot(range(len(cost_profile)), cost_profile, marker='o')
plt.xlabel("Time step k")
plt.ylabel("Cost-to-go $C[k,\\,s_0]$")
plt.title("Initial state cost-to-go over time (finite-horizon DP)")
plt.grid(True)
plt.tight_layout()
plt.show()
